In [63]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten,Dropout,Input
import pickle

In [14]:
datagen = ImageDataGenerator(rescale=1./255,fill_mode='constant',cval=0,data_format='channels_last')                                              

train_gen = datagen.flow_from_directory('AlzheimerDataset/train',class_mode='categorical',batch_size=16,target_size=(178,208))

test_gen = datagen.flow_from_directory('AlzheimerDataset/test',class_mode='categorical',batch_size=16,target_size=(178,208))

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [53]:
train_data,train_labels =  train_gen.next()
test_data,test_labels = test_gen.next()

In [54]:
#Making our own input tensor based on our image shape
input = Input(shape=(178,200,3),name = 'image_input')

#Using pretrained weights to help speed up training, only involves convolutional layers
model = tf.keras.applications.vgg16.VGG16(include_top=False,input_tensor=input, pooling = 'max') #added pooling

#Don't allow training 
model.trainable = True

#Create our fully connected structure
flatten_l = Flatten()(model.output)
fully_connected_one = Dense(1024,activation='relu', kernel_initializer='he_uniform')(flatten_l) 
dropout_one_fc = Dropout(0.25)(fully_connected_one)                                                   
output = Dense(4,activation='softmax')(dropout_one_fc)   

#Put our pretrained and custom model parts together
final = Model(inputs=model.inputs, outputs=output)

#Print a summary to look at model structure
final.summary()

#Use weights we already trained, comment out if you want to train it by yourself
final.load_weights('val20_epochs50.h5')

Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 178, 200, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 178, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 178, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 89, 100, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 89, 100, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 89, 100, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 44, 50, 128)     

In [55]:
#Use the scochastic gradient descent optimizer
opt = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=.9, nesterov=True,name='SGD') 

#Compile our model 
final.compile(optimizer=opt, loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [61]:
#Training model
#final.fit(train_data,train_labels,validation_data=(test_data,test_labels),epochs=5,batch_size=8,shuffle=True)

#Saving the weights so I can load them later if I want
final.save_weights('nick_attempt_4.h5')



In [57]:
#Check performance of model
final.evaluate(test_data,test_labels)

1/1 [==============================] - 0s 2ms/step - loss: 1.2099e-05 - accuracy: 1.0000


[1.209928814205341e-05, 1.0]

In [73]:
final.save('model_one.h5')